In [ ]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'

In [ ]:
from IPython import get_ipython
get_ipython().run_line_magic('run', 'setup')
import gspread
import df2gspread as d2g

gc = gspread.service_account()
sh = gc.open_by_key('1OOXPXubXqnsOdNckUvVqt5M6b6q9oMwJMKhSbK4BQyk')
ws = sh.get_worksheet(0)

def publish(df):
  sh.values_clear("'Regios'!A1:ZZ10000")
  sh.values_clear("'Regios'!A1:ZZ10000")
  ws.update([df.columns.values.tolist()] + df.values.tolist())

In [ ]:
@run('regios: download gemeenten en hun codes')
def cell():
  global gemeenten
  global bevolking
  gemeenten = pd.read_csv('gemeenten.csv')

  base = 'https://opendata.cbs.nl/ODataApi/OData/37230ned'

  # voor perioden pak de laatste
  periode = get_odata(base + '/Perioden').iloc[[-1]]['Key'].values[0]

  # startsWith would have been better to do in the filter but then the CBS "odata-ish" server responds with
  # "Object reference not set to an instance of an object."
  bevolking = get_odata(base + f"/TypedDataSet?$filter=(Perioden eq '{periode}')&$select=RegioS, BevolkingAanHetBeginVanDePeriode_1")
  bevolking = bevolking[bevolking.RegioS.str.startswith('GM')]
  bevolking.set_index('RegioS', inplace=True)
  # die _1 betekent waarschijnlijk dat het gedrag ooit gewijzigd is en er een nieuwe "versie" van die kolom is gepubliceerd
  bevolking.rename(columns = {'BevolkingAanHetBeginVanDePeriode_1': 'BevolkingAanHetBeginVanDePeriode'}, inplace = True)
  gemeenten = gemeenten.merge(bevolking, how='left', left_on='Code', right_index=True)
  gemeenten.loc[gemeenten.Personen == 0, 'Personen'] = gemeenten.BevolkingAanHetBeginVanDePeriode

  gemeenten = gemeenten[['Type', 'Landcode', 'GGD regio', 'Veiligheidsregio', 'Veiligheidsregio Code', 'Provincie', 'Landsdeel', 'Schoolregio', 'Personen', 'Opp land km2', 'Naam', 'Code']]
  publish(gemeenten.fillna(0))
  

In [ ]:
@run('regios: RIVM cijfers')
def cell():    
    global aantallen_gemeenten
    aantallen_gemeenten = rivm_cijfers('COVID-19_aantallen_gemeente_per_dag')
    # vervang lege gemeentecodes door de fallback 'GM0000'
    aantallen_gemeenten['Municipality_code'] = aantallen_gemeenten['Municipality_code'].fillna('GM0000')
    # knip de tijd van de datum af
    aantallen_gemeenten.Date_of_report = aantallen_gemeenten.Date_of_report.str.replace(' .*', '', regex=True)

    # simpele sum over groepering op gemeentecode, met rename
    positief_overleden = aantallen_gemeenten.groupby(['Municipality_code'])[['Total_reported', 'Deceased']].sum()
    positief_overleden.rename(columns={'Total_reported':'Positief getest', 'Deceased':'Overleden'}, inplace=True)
    
    # simpele sum over groepering op gemeentecode, met rename, voor records op de datum van publicatie
    positief_overleden_toename = aantallen_gemeenten[aantallen_gemeenten.Date_of_report == aantallen_gemeenten.Date_of_publication].groupby(['Municipality_code'])[['Total_reported', 'Deceased']].sum()
    positief_overleden_toename.rename(columns={'Total_reported':'Positief getest (toename)', 'Deceased':'Overleden (toename)'}, inplace=True)
    #Positief getest (toename)
    
    # al hetzelfde nog eens voor ziekenhuis opnames
    global ziekenhuisopnames
    ziekenhuisopnames = rivm_cijfers('COVID-19_ziekenhuisopnames')
    ziekenhuisopnames['Municipality_code'] = ziekenhuisopnames['Municipality_code'].fillna('GM0000')
    ziekenhuisopnames.Date_of_report = ziekenhuisopnames.Date_of_report.str.replace(' .*', '', regex=True)

    admissions = ziekenhuisopnames.groupby(['Municipality_code'])[['Hospital_admission']].sum()
    admissions.rename(columns={'Hospital_admission':'Ziekenhuisopname'}, inplace=True)
    admissions_toename = ziekenhuisopnames[ziekenhuisopnames.Date_of_report == ziekenhuisopnames.Date_of_statistics].groupby(['Municipality_code'])[['Hospital_admission']].sum()
    admissions_toename.rename(columns={'Hospital_admission':'Ziekenhuisopname (toename)'}, inplace=True)

    # en plak het zwik aan elkaar
    publish(
        gemeenten
        .merge(positief_overleden, how='left', left_on='Code', right_index=True)
        .merge(admissions, how='left', left_on='Code', right_index=True)
        .merge(positief_overleden_toename, how='left', left_on='Code', right_index=True)
        .merge(admissions_toename, how='left', left_on='Code', right_index=True)
        .fillna(0)
    )

In [ ]:
ziekenhuisopnames.groupby(['Date_of_statistics'])[['Hospital_admission']].sum()